In [1]:
import sys

import pandas as pd
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#% matplotlib inline
from datetime import datetime
import datefinder

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.express as px
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import iplot

from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')

import string
import gensim
import collections
from gensim.corpora import Dictionary

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

from autocorrect import Speller

from bs4 import BeautifulSoup
import requests
import csv
import time

import string
import gc
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx as nx
from itertools import chain

import matplotlib as mpl
from pylab import rcParams

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from sentence_transformers import SentenceTransformer
import hdbscan
import matplotlib.pyplot as plt

import umap

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package words is already up-to-date!


### Merging the collected json files from SerpAPI

##### Make a list of the files

In [2]:
# First listing out the names of all json files into a list
# as per the range of data files (related to page numbers)

files= []
string1 = 'serp_data/2012-2017/search_results_' # 2012-2017
string2 = '.json'

string3 = 'serp_data/2018-2019/search_results_' #2018-2019
string4 = 'serp_data/2020/search_results_'      #2020
string5 = 'serp_data/2021/search_results_'      #2021
string6 = 'serp_data/2022/search_results_'      #2022


for i in range(46):  # 2012-2017
    files.append(string1 + str(i) + string2)
    
for i in range(45):  #2018-2019
    files.append(string3 + str(i) + string2)
    
for i in range(30):  #2020
    files.append(string4 + str(i) + string2)
    
for i in range(48):  #2021
    files.append(string5 + str(i) + string2)
    
for i in range(35):  #2022
    files.append(string6 + str(i) + string2)    

##### Read and add them to a single pandas dataframe

In [3]:
# Create an empty dataframe and add the values from each json file of the list from it

data = pd.DataFrame()

for lst in files:
    with open(lst, 'r') as f:
        data_temp = json.load(f)
    data_temp = pd.json_normalize(data_temp, record_path =['organic_results'])
    data = pd.concat([data, data_temp], axis=0)  

In [4]:
data['index'] = range(1, len(data) + 1)
data

,position,title,result_id,type,link,snippet,resources,publication_info.summary,publication_info.authors,inline_links.serpapi_cite_link,...,inline_links.cited_by.serpapi_scholar_link,inline_links.related_pages_link,inline_links.serpapi_related_pages_link,inline_links.versions.total,inline_links.versions.link,inline_links.versions.cluster_id,inline_links.versions.serpapi_scholar_link,inline_links.cached_page_link,inline_links.html_version,index
0,0,Hateful symbols or hateful people? predictive ...,eG51Yln4CGsJ,Pdf,https://aclanthology.org/N16-2013.pdf,"… , NLP research on hate speech has been very ...","[{'title': 'aclanthology.org', 'file_format': ...","Z Waseem, D Hovy - Proceedings of the NAACL st...","[{'name': 'D Hovy', 'link': 'https://scholar.g...",https://serpapi.com/search.json?engine=google_...,...,https://serpapi.com/search.json?as_sdt=2000000...,https://scholar.google.com/scholar?q=related:e...,https://serpapi.com/search.json?as_sdt=0%2C21&...,7.0,https://scholar.google.com/scholar?cluster=771...,7712687424659615352,https://serpapi.com/search.json?as_sdt=0%2C21&...,https://scholar.googleusercontent.com/scholar?...,NaN,1
1,1,The Bullying Game: Sexism Based Toxic Language...,m6JOssuaA8UJ,NaN,https://search.proquest.com/openview/7e6d1aa29...,"… Based on this classification method, the mac...",NaN,"A Ekiciler, İ Ahioğlu, N Yıldırım, İ Ajas… - C...","[{'name': 'N Yıldırım', 'link': 'https://schol...",https://serpapi.com/search.json?engine=google_...,...,https://serpapi.com/search.json?as_sdt=2000000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2,Mean birds: Detecting aggression and bullying ...,nmwGhySbXz4J,NaN,https://dl.acm.org/doi/abs/10.1145/3091478.309...,… [36] perform hate speech detection on Yahoo ...,"[{'title': 'arxiv.org', 'file_format': 'PDF', ...","D Chatzakou, N Kourtellis, J Blackburn… - Proc...","[{'name': 'D Chatzakou', 'link': 'https://scho...",https://serpapi.com/search.json?engine=google_...,...,https://serpapi.com/search.json?as_sdt=2000000...,https://scholar.google.com/scholar?q=related:n...,https://serpapi.com/search.json?as_sdt=0%2C21&...,18.0,https://scholar.google.com/scholar?cluster=449...,4494481534325517470,https://serpapi.com/search.json?as_sdt=0%2C21&...,NaN,NaN,3
3,3,Hate is not binary: Studying abusive behavior ...,T3zAicbzZDkJ,NaN,https://dl.acm.org/doi/abs/10.1145/3078714.307...,… We perform an unsupervised machine learning ...,"[{'title': 'arxiv.org', 'file_format': 'PDF', ...","D Chatzakou, N Kourtellis, J Blackburn… - Proc...","[{'name': 'D Chatzakou', 'link': 'https://scho...",https://serpapi.com/search.json?engine=google_...,...,https://serpapi.com/search.json?as_sdt=2000000...,https://scholar.google.com/scholar?q=related:T...,https://serpapi.com/search.json?as_sdt=0%2C21&...,17.0,https://scholar.google.com/scholar?cluster=413...,4135698391873125455,https://serpapi.com/search.json?as_sdt=0%2C21&...,NaN,NaN,4
4,4,"On the Detection of Hate Speech, Hate Speakers...",qKtEpwXk55wJ,NaN,https://search.proquest.com/openview/395ac461a...,… of machine learning algorithms in understand...,"[{'title': 'cornell.edu', 'file_format': 'PDF'...",D Warmsley - 2017 - search.proquest.com,"[{'name': 'D Warmsley', 'link': 'https://schol...",https://serpapi.com/search.json?engine=google_...,...,https://serpapi.com/search.json?as_sdt=2000000...,https://scholar.google.com/scholar?q=related:q...,https://serpapi.com/search.json?as_sdt=0%2C21&...,2.0,https://scholar.google.com/scholar?cluster=113...,11306256102469184424,https://serpapi.com/search.json?as_sdt=0%2C21&...,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,14,Peer Support IDD Scoping Review Search,vHK6O-kNJ9EJ,NaN,https://scholarshare.temple.edu/handle/20.500....,To identify studies to include or consider for...,"[{'title': 'temple.edu', 'file_format': 'HTML'...","S Roth, B Pfeiffer, KE Weiss, S Aleong, L Karp...","[{'name': 'S Roth', 'link': 'https://scholar.g...",https://serpapi.com/search.json?engine=

In [5]:
data['index'].unique() # no. of results

array([   1,    2,    3, ..., 4027, 4028, 4029])

In [6]:
data.columns

Index(['position', 'title', 'result_id', 'type', 'link', 'snippet',
       'resources', 'publication_info.summary', 'publication_info.authors',
       'inline_links.serpapi_cite_link', 'inline_links.cited_by.total',
       'inline_links.cited_by.link', 'inline_links.cited_by.cites_id',
       'inline_links.cited_by.serpapi_scholar_link',
       'inline_links.related_pages_link',
       'inline_links.serpapi_related_pages_link',
       'inline_links.versions.total', 'inline_links.versions.link',
       'inline_links.versions.cluster_id',
       'inline_links.versions.serpapi_scholar_link',
       'inline_links.cached_page_link', 'inline_links.html_version', 'index'],
      dtype='object')

In [7]:
data['resources'].isna().sum() # A lot of missing values, so cannot use that

1569

In [8]:
data['publication_info.summary']

0     Z Waseem, D Hovy - Proceedings of the NAACL st...
1     A Ekiciler, İ Ahioğlu, N Yıldırım, İ Ajas… - C...
2     D Chatzakou, N Kourtellis, J Blackburn… - Proc...
3     D Chatzakou, N Kourtellis, J Blackburn… - Proc...
4               D Warmsley - 2017 - search.proquest.com
                            ...                        
14    S Roth, B Pfeiffer, KE Weiss, S Aleong, L Karp...
15    A Thomson, S Ganz - Margaret Atwood Studies, 2...
16    V Ceia, B Nothwehr, L Wagner - webassets.oxfam...
17                 S Behan - 2022 - search.proquest.com
18                  C CHUN - 2022 - search.proquest.com
Name: publication_info.summary, Length: 4029, dtype: object

In [9]:
def substring_after(s, delim):
    return s.partition(delim)[2]

#def substring_before(s, delim):
#    return s.partition(delim)[0]

def substring_before_date(s):
    #date = re.search(r"\d{4}", s).group(0)
    date = re.findall('(\d{4})', s)
    if bool(date): # checks if the list is empty or not
        if date[0] in s:
            value = s.partition(date[0])[0]
        elif s.startswith(date[0]):
            value = ''
    else:
        value = ''
    return value

In [10]:
data['extracted_public_inf'] = data['publication_info.summary'].apply(lambda x: substring_after(x, '-'))

## Observations:
# Multiple publications leading to the same website

#data['extracted_public_inf'][2], data['extracted_public_inf'][3]

In [11]:
test = data['extracted_public_inf'].iloc[8]
print(substring_before_date(test))

 Minn. L. Rev., 


In [12]:
data.index = data['index']
data['extracted_public_inf']

index
1        Proceedings of the NAACL student research …, ...
2        Conference on Gender …, 2014 - search.proques...
3                 Proceedings of the …, 2017 - dl.acm.org
4            Proceedings of the 28th …, 2017 - dl.acm.org
5                              2017 - search.proquest.com
                              ...                        
4025                       2022 - scholarshare.temple.edu
4026     Margaret Atwood Studies, 2022 - search.proque...
4027                           webassets.oxfamamerica.org
4028                           2022 - search.proquest.com
4029                           2022 - search.proquest.com
Name: extracted_public_inf, Length: 4029, dtype: object

In [13]:
data['extracted_public_inf'] = data['extracted_public_inf'].apply(lambda x: substring_before_date(x))
data['extracted_public_inf']

index
1        Proceedings of the NAACL student research …, 
2                             Conference on Gender …, 
3                               Proceedings of the …, 
4                          Proceedings of the 28th …, 
5                                                     
                             ...                      
4025                                                  
4026                         Margaret Atwood Studies, 
4027                                                  
4028                                                  
4029                                                  
Name: extracted_public_inf, Length: 4029, dtype: object

In [14]:
# to remove whitespace and characters from the beginning and the end of a string\n",
data['extracted_public_inf'] = data['extracted_public_inf'].str.strip()

# Apply a function to clean the data and remove the unnecessary characters
def clean_text(x):
    text = x.replace(r',', '') 
    #text = text.replace(r'...', '') 
    text = text.lstrip('./')
    return text

data['extracted_public_inf'] = data['extracted_public_inf'].apply(clean_text)

### Understanding the data

In [15]:
data['resources'] ## can see where from we get the data

index
1       [{'title': 'aclanthology.org', 'file_format': ...
2                                                     NaN
3       [{'title': 'arxiv.org', 'file_format': 'PDF', ...
4       [{'title': 'arxiv.org', 'file_format': 'PDF', ...
5       [{'title': 'cornell.edu', 'file_format': 'PDF'...
                              ...                        
4025    [{'title': 'temple.edu', 'file_format': 'HTML'...
4026                                                  NaN
4027    [{'title': 'oxfamamerica.org', 'file_format': ...
4028                                                  NaN
4029                                                  NaN
Name: resources, Length: 4029, dtype: object

In [16]:
# Get the source page titles from the nested json
sources = pd.json_normalize(data['resources'])
nested_sources = pd.json_normalize(sources[0])
data['source_title']= nested_sources['title']
data['source_title']

index
1                    NaN
2              arxiv.org
3              arxiv.org
4            cornell.edu
5               unisa.it
              ...       
4025                 NaN
4026    oxfamamerica.org
4027                 NaN
4028                 NaN
4029                 NaN
Name: source_title, Length: 4029, dtype: object